In [5]:
from helpers import load_data
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier # pip install imblearn - scikit wrapper
from sklearn import ensemble, pipeline, model_selection, metrics, preprocessing, svm
import matplotlib.pyplot as plt
import pandas as pd

DATA_PATH = '../data/'
seed = 7

data = load_data(DATA_PATH)
X, y = data['muchlinski']

In [ ]:
kernels = ['linear','poly','rbf','sigmoid','precomputed']
k_folds = 5
degrees = np.arange(1,3)
class_weights = [{0:1, 1:2**i} for i in range(3)]
penalties = np.logspace(-5, 3, 3)
# degrees = np.arange(1,8)
# class_weights = [{0:1, 1:2**i} for i in range(8)]
# penalties = np.logspace(-5, 3, 9)

res = []

scoring=['precision','recall','f1','roc_auc']

for kernel in kernels:
    for class_weight in class_weights:
        for penalty in penalties: 
            if kernel=='poly':
                # if polynomial kernel try multiple degrees
                for d in degrees:
                    # define pipeline and cross validate
                    pipe = pipeline.make_pipeline(preprocessing.StandardScaler(), svm.SVC(random_state=seed, class_weight=class_weight, C=penalty, kernel=kernel))
                    scores = model_selection.cross_validate(pipe, X=X, y=y, cv=k_folds, scoring=scoring, return_train_score=False)

                    # create backlog of results
                    cv_res = [kernel, class_weight[1], penalty, d, np.mean(scores['test_precision']), np.mean(scores['test_recall']), np.mean(scores['test_f1']), np.mean(scores['test_roc_auc'])] 
                    res.append(cv_res)
            else:
                # define pipeline and cross validate
                pipe = pipeline.make_pipeline(preprocessing.StandardScaler(), svm.SVC(random_state=seed, class_weight=class_weight, C=penalty, kernel=kernel))
                scores = model_selection.cross_validate(pipe, X=X, y=y, cv=k_folds, scoring=scoring, return_train_score=False)

                # create backlog of results
                cv_res = [kernel, class_weight[1], penalty, d, np.mean(scores['test_precision']), np.mean(scores['test_recall']), np.mean(scores['test_f1']), np.mean(scores['test_roc_auc'])] 
                res.append(cv_res)
        
res = pd.DataFrame(np.vstack(res), columns=['n_tree','sample_proportion','prec','rec','f1','roc_auc'])